In [85]:
import networkx as nx
import numpy as np
from scipy.optimize import linprog

In [119]:
# Step 1: Create the MultiGraph with nodes and edges
m = nx.MultiGraph()
m.add_node("A", port_capacity=400)
m.add_node("B", port_capacity=400)
m.add_node("C", port_capacity=100)

m.add_edge("A", "B", priority=3)
m.add_edge("A", "C", priority=2)
m.add_edge("B", "C", priority=1)
m.add_edge("B", "C", priority=1)

1

In [120]:
# Step 2: Convert MultiGraph to a simple Graph with summed priorities
g = nx.Graph()
g.add_nodes_from(m.nodes(data=True))

for u, v, data in m.edges(data=True):
    priority = data['priority']
    if g.has_edge(u, v):
        g[u][v]['priority'] += priority
    else:
        g.add_edge(u, v, priority=priority)

In [121]:
g = nx.Graph()
g.add_nodes_from(m.nodes(data=True))

for u, v, data in m.edges(data=True):
    priority = data['priority']
    if g.has_edge(u, v):
        g[u][v]['priority'] += priority
    else:
        g.add_edge(u, v, priority=priority)

nodes = list(g.nodes)
edges = list(g.edges(data=True))

n_nodes = len(nodes)
n_edges = len(edges)

In [124]:
node_index = {node: i for i, node in enumerate(nodes)}
edge_index = {edge[:2]: i for i, edge in enumerate(edges)}

A = np.zeros((n_nodes, n_edges))
c = np.zeros(n_edges)

for (u, v, data) in edges:
    i = node_index[u]
    j = node_index[v]
    priority = data['priority']
    edge_idx = edge_index[(u, v)]

    A[i, edge_idx] = priority  # Outgoing edge from node i
    A[j, edge_idx] = priority  # Incoming edge to node j
    c[edge_idx] = -priority    # We minimize -priority to maximize flow

b = np.array([g.nodes[node]['port_capacity'] for node in nodes])

optim_result = linprog(c, A_ub=A, b_ub=b, bounds=(0, None))

if optim_result.success:
    x = optim_result.x  # Optimal flow on each edge
    final_flow = x * np.array([data['priority'] for u, v, data in edges])
    
    for u, v, key, data in m.edges(keys=True, data=True):
        total_priority = g[u][v]['priority']  # Get the summed priority in the simple graph
        if total_priority > 0:
            proportion = data['priority'] / total_priority
            final_flow_value = final_flow[edge_index[(u, v)]] * proportion
            m[u][v][key]['bandwidth'] = final_flow_value
        else:
            m[u][v][key]['bandwidth'] = 0

else:
    print("Optimization failed.")


In [123]:
m.edges(data=True)

MultiEdgeDataView([('A', 'B', {'priority': 3, 'bandwidth': 350.0}), ('A', 'C', {'priority': 2, 'bandwidth': 50.0}), ('B', 'C', {'priority': 1, 'bandwidth': 25.0}), ('B', 'C', {'priority': 1, 'bandwidth': 25.0})])